In [70]:
import requests
from bs4 import BeautifulSoup
import time
import datetime
import MySQLdb
all_activities_url = []
page = 1
domain = 'https://streetvoice.com/venue/activities/all/0/'

def get_content(domain, selector, page=1):
    query_string = {
        'page': page
    }
    html = requests.get(domain, params=query_string).text
    soup = BeautifulSoup(html)
    return soup.select(selector)

html_activities = get_content(domain, '.live-item-info')

while len(html_activities) > 0:
    for activity in html_activities:
        url = activity.select('h3 a', href=True)[0]["href"]
        all_activities_url.append(f'https://streetvoice.com{url}')
    page += 1
    html_activities = get_content(domain, '.live-item-info', page)
    time.sleep(5)

In [76]:
#SQL connection data to connect and save the data in
HOST = "114.34.237.146"
USERNAME = "root"
PASSWORD = ""
DATABASE = "Bump"
PORT = 3000

In [197]:
def to_timestamp(time_list):
    YMD_list = time_list[0].split(' ')
    dt_local = datetime.datetime.strptime(f'{YMD_list[0]}-{YMD_list[2].zfill(2)}-{YMD_list[4].zfill(2)} {time_list[2]}:00','%Y-%m-%d %H:%M:%S')
    ts = time.mktime(dt_local.timetuple()) + 1e-6 * dt_local.microsecond
    dt_utc = datetime.datetime.utcfromtimestamp(ts)
    return str(dt_utc)

tmp_urls = ['https://streetvoice.com/venue/activities/903/', 'https://streetvoice.com/venue/activities/904/', 'https://streetvoice.com/venue/activities/905/']

activity_database = []

for url in tmp_urls:
    soup_tmp = BeautifulSoup(requests.get(url).text).select('#pjax-container')[0]
    activity_database.append({
        'title': soup_tmp.select('h1')[0].text,
        'artist': soup_tmp.select('h3:nth-of-type(4)')[0].text.strip(),
        'startTime': to_timestamp(soup_tmp.select('h3:nth-of-type(1)')[0].text.strip().split(' l ')),
        'website': url,
        'photo': soup_tmp.select('img')[0]['src'],
        'describe': str(soup_tmp.select('.article-block')[0]),
        'location': soup_tmp.select('h3:nth-of-type(2)')[0].text.strip().split('\n')[0].split('．')[1]
    })
    time.sleep(3)

[{'title': '《南國再見，南國》秘密集會VOL.1：憨孫仔 午場 ＠台北場新店飄香麵館', 'artist': 'Theseus忒修斯', 'startTime': '2020-05-09 04:00:00', 'website': 'https://streetvoice.com/venue/activities/903/', 'photo': 'https://cfstatic.streetvoice.com/venue_activity_image/2020/03/12/99f0ea7629de4930a4c8981cd3908f72.jpg?x-oss-process=image/resize,m_fill,h_440,w_830,limit_0/interlace,1/quality,q_85/format,jpg', 'describe': '<div class="text-read article-block mb-5">\n<p>完成《南國再見，南國》後，<br/>我們對於歌曲故事如何講述，持續苦惱著。<br/>深怕只透過巡演，依然木訥的我們，<br/>能不能娓娓道來這兩年的時光，<br/>又將如此珍貴的交往，變得太過輕巧，<br/><br/>『還是吃一頓飯，好好坐著聊聊天吧。』<br/><br/>於是，我們決定在台北、台中、高雄，<br/>分別對應EP曲目中的三首歌曲，<br/>選擇特別的場所，舉辦「秘密集會｜食唱會」<br/><br/>『活動結束前，我們不會把整張作品數位上架！』<br/><br/>我們希望，能夠與妳/你一起好好聆聽，<br/>我們銘記的痛苦與美好記憶，好好聚一聚。<br/>第一場演出，直接從「家裏」開始唱。<br/><br/>我們選擇主唱小正家的牛肉麵館「飄香麵館」<br/>一個承載二十多年，成長記憶的地方開始，<br/>與台北聽眾們一同品嚐家中最溫暖的味道。<br/><br/>這場集會邀請每一個被迫長大，<br/>卻仍共享成長回憶的妳/你，<br/>一起來到主唱小正奶奶開的牛肉麵店，<br/>享用當日忒餐：飄香祖傳忒別版牛肉麵。<br/>讓我們一同歌唱，<br/>用奶奶的手藝與家鄉味溫暖心房。<br/><br/>- - - -<br/>地址：新北市新店區寶安街31號（近大坪林捷運站）<br/><br/>1.當日進場

In [198]:
db = MySQLdb.connect(host=HOST, user=USERNAME, db=DATABASE, port=PORT)
# prepare a cursor object using cursor() method
cursor = db.cursor()
table = 'GigSource_copy'
columns = ', '.join(list(map(lambda prop: f'`{prop}`', activity_database[0].keys())))
placeholders = ", ".join(['{}'] * len(activity_database[0]))
sql = "insert into {table} ({columns}) values ({values});".format(table=table, columns=columns, values=placeholders)

for activity in activity_database:
    sql_string = sql.format(
        repr(activity.get('title')),
        repr(activity.get('artist')),
        repr(activity.get('startTime')), 
        repr(activity.get('website')), 
        repr(activity.get('photo')), 
        repr(activity.get('describe')), 
        repr(activity.get('location'))
    )
    try:
        db.ping(True)
        # Execute the SQL command
        cursor.execute(sql_string)
        # Commit your changes in the database
        db.commit()
    except:
        # Rollback in case there is any error
        db.rollback()
        # disconnect from server
        db.close()